In [1]:
import pymysql
import pandas as pd
import config
from concurrent.futures import ThreadPoolExecutor, as_completed
import logging

# Configurazioni dal file config
DB_USER = config.DB_USER
DB_PASSWORD = config.DB_PASSWORD
DB_NAME = config.DB_NAME

# Parametri specifici
lookback_hours = 24  # Ore di lookback

# Configura il logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def process_device(id_device, db_user, db_password, db_name, lookback_hours):
    logging.info(f"Inizio elaborazione dispositivo {id_device}")
    try:
        with pymysql.connect(
            host='54.195.165.244',
            user=db_user,
            password=db_password,
            database=db_name,
            port=3306
        ) as connection:
            with connection.cursor() as cursor:
                # Esegui la query per ottenere tutti gli eventi per il dispositivo specificato
                event_query = """
                SELECT 
                    de.id AS event_id,
                    de.id_device,
                    de.date_start,
                    de.date_end,
                    de.variable AS event_variable,
                    de.operator,
                    de.reference_value,
                    d.name AS device_name
                FROM device_events de
                JOIN devices d ON de.id_device = d.id
                WHERE de.id_device = %s
                ORDER BY de.date_start;
                """
                cursor.execute(event_query, (id_device,))
                events = cursor.fetchall()

                # Lista per memorizzare i DataFrame delle misure
                all_measures_dataframes = []

                for event in events:
                    event_id = event[0]
                    date_start = event[2]
                    operator_value = event[5]

                    # Calcola il periodo di 24 ore prima dell'evento
                    lookback_start = pd.to_datetime(date_start) - pd.Timedelta(hours=lookback_hours)

                    # Esegui la query per ottenere tutte le misure delle 24 ore precedenti l'evento
                    measure_query = """
                    SELECT 
                        m.id AS measure_id,
                        m.oid,
                        m.name AS measure_name,
                        m.value,
                        m.value_label,
                        m.measure_date,
                        d.name AS device_name,
                        %s AS event_id,
                        %s AS event_variable,
                        %s AS event_operator,
                        %s AS event_reference_value
                    FROM measures m
                    JOIN devices d ON m.id_device = d.id
                    WHERE m.id_device = %s
                      AND m.measure_date BETWEEN %s AND %s
                    ORDER BY m.measure_date;
                    """
                    cursor.execute(measure_query, (event_id, event[4], event[5], event[6], id_device, lookback_start, date_start))
                    measures = cursor.fetchall()

                    # Se non ci sono misure per questo evento, continua con il prossimo
                    if not measures:
                        logging.info(f"Nessuna misura trovata per l'evento {event_id} nelle 24 ore precedenti.")
                        continue

                    # Converti le misure in DataFrame
                    measure_columns = ['measure_id', 'oid', 'measure_name', 'value', 'value_label', 'measure_date', 'device_name', 'event_id', 'event_variable', 'event_operator', 'event_reference_value']
                    measures_df = pd.DataFrame(measures, columns=measure_columns)

                    # Aggiungi il DataFrame alla lista
                    all_measures_dataframes.append(measures_df)

                # Controlla se ci sono DataFrame da concatenare
                if all_measures_dataframes:
                    # Combina tutti i DataFrame in uno solo
                    combined_df = pd.concat(all_measures_dataframes, ignore_index=True)

                    # Salva il DataFrame combinato in un file CSV per il dispositivo
                    combined_csv_path = f'csv/measures_24h_before_events_device_{id_device}.csv'
                    combined_df.to_csv(combined_csv_path, index=False)
                    logging.info(f"Tutte le misure per le 24 ore precedenti gli eventi del dispositivo {id_device} sono state salvate in {combined_csv_path}")
                else:
                    logging.info(f"Nessuna misura trovata per le 24 ore precedenti gli eventi del dispositivo {id_device}.")
    except pymysql.MySQLError as e:
        logging.error(f"Errore durante l'esecuzione della query: {e}")
    except Exception as e:
        logging.error(f"Errore generico: {e}")

def main():
    try:
        connection = pymysql.connect(
            host='54.195.165.244',
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME,
            port=3306  # Assicurati di specificare la porta
        )
        with connection.cursor() as cursor:
            # Ottenere l'elenco unico dei dispositivi
            cursor.execute("SELECT DISTINCT id_device FROM device_events")
            devices = cursor.fetchall()

        # Parallelizza l'elaborazione dei dispositivi
        with ThreadPoolExecutor(max_workers=16) as executor:
            futures = [executor.submit(process_device, device[0], DB_USER, DB_PASSWORD, DB_NAME, lookback_hours) for device in devices]
            for future in as_completed(futures):
                future.result()

    except pymysql.MySQLError as e:
        logging.error(f"Errore durante l'esecuzione della query: {e}")
    finally:
        connection.close()

if __name__ == "__main__":
    main()


2024-07-11 22:58:38,393 - INFO - Inizio elaborazione dispositivo 78
2024-07-11 22:58:38,393 - INFO - Inizio elaborazione dispositivo 84
2024-07-11 22:58:38,394 - INFO - Inizio elaborazione dispositivo 116
2024-07-11 22:58:38,394 - INFO - Inizio elaborazione dispositivo 546
2024-07-11 22:58:38,394 - INFO - Inizio elaborazione dispositivo 631
2024-07-11 22:58:38,394 - INFO - Inizio elaborazione dispositivo 632
2024-07-11 23:01:11,560 - INFO - Tutte le misure per le 24 ore precedenti gli eventi del dispositivo 632 sono state salvate in csv/measures_24h_before_events_device_632.csv
2024-07-11 23:01:11,839 - INFO - Tutte le misure per le 24 ore precedenti gli eventi del dispositivo 84 sono state salvate in csv/measures_24h_before_events_device_84.csv
2024-07-11 23:01:11,848 - INFO - Tutte le misure per le 24 ore precedenti gli eventi del dispositivo 116 sono state salvate in csv/measures_24h_before_events_device_116.csv
2024-07-11 23:23:01,779 - INFO - Tutte le misure per le 24 ore preceden

In [2]:
import pymysql
import pandas as pd
import config
from concurrent.futures import ThreadPoolExecutor, as_completed
import logging
from queue import Queue
from threading import Lock
import time

# Configurations from the config file
DB_USER = config.DB_USER
DB_PASSWORD = config.DB_PASSWORD
DB_NAME = config.DB_NAME

# Specific parameters
lookback_hours = 24  # Lookback hours

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Connection pool settings
POOL_SIZE = 16
connection_pool = Queue(maxsize=POOL_SIZE)
pool_lock = Lock()

# Retry settings
MAX_RETRIES = 5
RETRY_BACKOFF = 2

def create_connection():
    return pymysql.connect(
        host='54.195.165.244',
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
        port=3306
    )

def initialize_connection_pool():
    for _ in range(POOL_SIZE):
        connection_pool.put(create_connection())

def get_connection():
    with pool_lock:
        if connection_pool.empty():
            return create_connection()
        else:
            return connection_pool.get()

def release_connection(connection):
    with pool_lock:
        if connection_pool.full():
            connection.close()
        else:
            connection_pool.put(connection)

def fetch_data(query, params):
    """Helper function to execute a query and fetch all results with retry logic."""
    for attempt in range(MAX_RETRIES):
        connection = get_connection()
        try:
            with connection.cursor() as cursor:
                cursor.execute(query, params)
                return cursor.fetchall()
        except pymysql.MySQLError as e:
            logging.error(f"Errore durante l'esecuzione della query (tentativo {attempt + 1}): {e}")
            if e.args[0] in (2006, 2013, 2014, 2015):
                if attempt < MAX_RETRIES - 1:
                    time.sleep(RETRY_BACKOFF ** attempt)  # Backoff esponenziale
                    continue  # Retry
                else:
                    raise  # Raggiunto il numero massimo di tentativi, solleva l'eccezione
        finally:
            release_connection(connection)

def save_to_csv(df, path):
    """Helper function to save a DataFrame to a CSV file."""
    df.to_csv(path, index=False)
    logging.info(f"Dati salvati in {path}")

def process_model(model_id, model_name):
    logging.info(f"Analizzando il modello: {model_name}")

    event_query = """
    SELECT 
        de.id AS event_id,
        de.id_device,
        de.date_start,
        de.date_end,
        de.variable AS event_variable,
        de.operator,
        de.reference_value,
        d.name AS device_name
    FROM device_events de
    JOIN devices d ON de.id_device = d.id
    JOIN device_models dm ON d.id_device_model = dm.id
    WHERE dm.id = %s
    ORDER BY de.date_start;
    """
    events = fetch_data(event_query, (model_id,))

    if not events:
        logging.info(f"Nessun evento trovato per il modello {model_name}.")
        return

    measures_dataframes = []

    for event in events:
        event_id, id_device, date_start = event[0], event[1], event[2]
        lookback_start = pd.to_datetime(date_start) - pd.Timedelta(hours=lookback_hours)

        measure_query = """
        SELECT 
            m.id AS measure_id,
            m.oid,
            m.name AS measure_name,
            m.value,
            m.value_label,
            m.measure_date,
            d.name AS device_name
        FROM measures m
        JOIN devices d ON m.id_device = d.id
        WHERE m.id_device = %s
          AND m.measure_date BETWEEN %s AND %s
        ORDER BY m.measure_date;
        """
        measures = fetch_data(measure_query, (id_device, lookback_start, date_start))

        if not measures:
            logging.info(f"Nessuna misura trovata per l'evento {event_id} nelle 24 ore precedenti.")
            continue

        measure_columns = ['measure_idå©', 'oid', 'measure_name', 'value', 'value_label', 'measure_date', 'device_name']
        measures_df = pd.DataFrame(measures, columns=measure_columns)

        measures_df['event_id'] = event_id
        measures_df['event_variable'] = event[4]
        measures_df['event_operator'] = event[5]
        measures_df['event_reference_value'] = event[6]

        output_csv_path = f'csv/measures_24h_before_event_{event_id}_model_{model_name}.csv'
        save_to_csv(measures_df, output_csv_path)
        measures_dataframes.append(measures_df)

    if measures_dataframes:
        combined_df = pd.concat(measures_dataframes, ignore_index=True)
        combined_csv_path = f'csv/measures_24h_before_events_for_model_{model_name}.csv'
        save_to_csv(combined_df, combined_csv_path)
    else:
        logging.info(f"Nessuna misura trovata per le 24 ore precedenti gli eventi per il modello {model_name}.")

def main():
    try:
        initialize_connection_pool()
        models = fetch_data("SELECT id, model FROM device_models;", ())

        # Parallelize the processing of models
        with ThreadPoolExecutor(max_workers=POOL_SIZE) as executor:
            futures = [executor.submit(process_model, model_id, model_name) for model_id, model_name in models]
            for future in as_completed(futures):
                try:
                    future.result()
                except Exception as e:
                    logging.error(f"Errore durante l'elaborazione: {e}")

    except pymysql.MySQLError as e:
        logging.error(f"Errore durante l'esecuzione della query: {e}")

if __name__ == "__main__":
    main()


2024-07-12 11:33:51,939 - INFO - Analizzando il modello: Combo
2024-07-12 11:33:51,939 - INFO - Analizzando il modello: OneDriver
2024-07-12 11:33:51,940 - INFO - Analizzando il modello: TemDab
2024-07-12 11:33:51,940 - INFO - Analizzando il modello: RX 2 GHz
2024-07-12 11:33:51,941 - INFO - Analizzando il modello: TX 2 GHz
2024-07-12 11:33:51,941 - INFO - Analizzando il modello: General
2024-07-12 11:33:51,942 - INFO - Analizzando il modello: TxDab
2024-07-12 11:33:51,942 - INFO - Analizzando il modello: ALPAN DVB-T
2024-07-12 11:33:51,943 - INFO - Analizzando il modello: CCU 1+1
2024-07-12 11:33:51,943 - INFO - Analizzando il modello: DVB-T2
2024-07-12 11:33:51,944 - INFO - Analizzando il modello: DVBT
2024-07-12 11:33:51,944 - INFO - Analizzando il modello: IEC DVB-T
2024-07-12 11:33:51,944 - INFO - Analizzando il modello: dab
2024-07-12 11:33:51,945 - INFO - Analizzando il modello: DAB
2024-07-12 11:33:51,945 - INFO - Analizzando il modello: IEC DVB-T2
2024-07-12 11:33:51,946 - INF

In [1]:
import pymysql
import pandas as pd
import config
from concurrent.futures import ThreadPoolExecutor, as_completed
import logging
from queue import Queue
from threading import Lock
import time
import os
import json

# Configurations from the config file
DB_USER = config.DB_USER
DB_PASSWORD = config.DB_PASSWORD
DB_NAME = config.DB_NAME

# Specific parameters
lookback_hours = 24  # Lookback hours

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Connection pool settings
POOL_SIZE = 16
connection_pool = Queue(maxsize=POOL_SIZE)
pool_lock = Lock()

# Retry settings
MAX_RETRIES = 5
RETRY_BACKOFF = 2

STATE_FILE = 'state.json'

def create_connection():
    return pymysql.connect(
        host='54.195.165.244',
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
        port=3306
    )

def initialize_connection_pool():
    for _ in range(POOL_SIZE):
        connection_pool.put(create_connection())

def get_connection():
    with pool_lock:
        if connection_pool.empty():
            return create_connection()
        else:
            return connection_pool.get()

def release_connection(connection):
    with pool_lock:
        if connection_pool.full():
            connection.close()
        else:
            connection_pool.put(connection)

def fetch_data(query, params):
    """Helper function to execute a query and fetch all results with retry logic."""
    for attempt in range(MAX_RETRIES):
        connection = get_connection()
        logging.info(f"Uso della connessione: {connection}")
        try:
            with connection.cursor() as cursor:
                cursor.execute(query, params)
                return cursor.fetchall()
        except pymysql.MySQLError as e:
            logging.error(f"Errore durante l'esecuzione della query (tentativo {attempt + 1}): {e}")
            if e.args[0] in (2006, 2013, 2014, 2015):
                if attempt < MAX_RETRIES - 1:
                    time.sleep(RETRY_BACKOFF ** attempt)  # Backoff esponenziale
                    continue  # Retry
                else:
                    raise  # Raggiunto il numero massimo di tentativi, solleva l'eccezione
        finally:
            release_connection(connection)

def save_to_csv(df, path):
    """Helper function to save a DataFrame to a CSV file."""
    df.to_csv(path, index=False)
    logging.info(f"Dati salvati in {path}")

def load_state():
    if os.path.exists(STATE_FILE):
        with open(STATE_FILE, 'r') as f:
            return json.load(f)
    return {}

def save_state(state):
    with open(STATE_FILE, 'w') as f:
        json.dump(state, f)

def process_model(model_id, model_name, state):
    if state.get(model_name) == 'completed':
        logging.info(f"Modello {model_name} già elaborato. Salto.")
        return

    logging.info(f"Analizzando il modello: {model_name}")

    event_query = """
    SELECT 
        de.id AS event_id,
        de.id_device,
        de.date_start,
        de.date_end,
        de.variable AS event_variable,
        de.operator,
        de.reference_value,
        d.name AS device_name
    FROM device_events de
    JOIN devices d ON de.id_device = d.id
    JOIN device_models dm ON d.id_device_model = dm.id
    WHERE dm.id = %s
    ORDER BY de.date_start;
    """
    events = fetch_data(event_query, (model_id,))

    if not events:
        logging.info(f"Nessun evento trovato per il modello {model_name}.")
        state[model_name] = 'completed'
        save_state(state)
        return

    measures_dataframes = []

    for event in events:
        event_id, id_device, date_start = event[0], event[1], event[2]
        lookback_start = pd.to_datetime(date_start) - pd.Timedelta(hours=lookback_hours)

        measure_query = """
        SELECT 
            m.id AS measure_id,
            m.oid,
            m.name AS measure_name,
            m.value,
            m.value_label,
            m.measure_date,
            d.name AS device_name
        FROM measures m
        JOIN devices d ON m.id_device = d.id
        WHERE m.id_device = %s
          AND m.measure_date BETWEEN %s AND %s
        ORDER BY m.measure_date;
        """
        measures = fetch_data(measure_query, (id_device, lookback_start, date_start))

        if not measures:
            logging.info(f"Nessuna misura trovata per l'evento {event_id} nelle 24 ore precedenti.")
            continue

        measure_columns = ['measure_id', 'oid', 'measure_name', 'value', 'value_label', 'measure_date', 'device_name']
        measures_df = pd.DataFrame(measures, columns=measure_columns)

        measures_df['event_id'] = event_id
        measures_df['event_variable'] = event[4]
        measures_df['event_operator'] = event[5]
        measures_df['event_reference_value'] = event[6]

        output_csv_path = f'csv/measures_24h_before_event_{event_id}_model_{model_name}.csv'
        save_to_csv(measures_df, output_csv_path)
        measures_dataframes.append(measures_df)

    if measures_dataframes:
        combined_df = pd.concat(measures_dataframes, ignore_index=True)
        combined_csv_path = f'csv/measures_24h_before_events_for_model_{model_name}.csv'
        save_to_csv(combined_df, combined_csv_path)
    else:
        logging.info(f"Nessuna misura trovata per le 24 ore precedenti gli eventi per il modello {model_name}.")

    state[model_name] = 'completed'
    save_state(state)

def main():
    state = load_state()
    try:
        initialize_connection_pool()
        models = fetch_data("SELECT id, model FROM device_models;", ())

        # Parallelize the processing of models
        with ThreadPoolExecutor(max_workers=POOL_SIZE) as executor:
            futures = [executor.submit(process_model, model_id, model_name, state) for model_id, model_name in models]
            for future in as_completed(futures):
                try:
                    future.result()
                except Exception as e:
                    logging.error(f"Errore durante l'elaborazione: {e}")

    except pymysql.MySQLError as e:
        logging.error(f"Errore durante l'esecuzione della query: {e}")

if __name__ == "__main__":
    main()


2024-07-12 11:46:57,645 - INFO - Uso della connessione: <pymysql.connections.Connection object at 0x1319d1ee0>
2024-07-12 11:46:57,723 - INFO - Modello Combo già elaborato. Salto.
2024-07-12 11:46:57,723 - INFO - Modello OneDriver già elaborato. Salto.
2024-07-12 11:46:57,724 - INFO - Modello TemDab già elaborato. Salto.
2024-07-12 11:46:57,724 - INFO - Analizzando il modello: RX 2 GHz
2024-07-12 11:46:57,726 - INFO - Modello TX 2 GHz già elaborato. Salto.
2024-07-12 11:46:57,726 - INFO - Modello General già elaborato. Salto.
2024-07-12 11:46:57,726 - INFO - Modello TxDab già elaborato. Salto.
2024-07-12 11:46:57,727 - INFO - Modello ALPAN DVB-T già elaborato. Salto.
2024-07-12 11:46:57,727 - INFO - Modello CCU 1+1 già elaborato. Salto.
2024-07-12 11:46:57,727 - INFO - Modello DVB-T2 già elaborato. Salto.
2024-07-12 11:46:57,727 - INFO - Modello DVBT già elaborato. Salto.
2024-07-12 11:46:57,728 - INFO - Modello IEC DVB-T già elaborato. Salto.
2024-07-12 11:46:57,728 - INFO - Modello d

In [3]:
import pymysql
import pandas as pd
import config
from concurrent.futures import ThreadPoolExecutor, as_completed
import logging

# Configurazioni dal file config
DB_USER = config.DB_USER
DB_PASSWORD = config.DB_PASSWORD
DB_NAME = config.DB_NAME

# Configura il logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Funzione per connettersi al database
def connect_to_database():
    try:
        return pymysql.connect(
            host='54.195.165.244',
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME,
            port=3306
        )
    except pymysql.MySQLError as e:
        logging.error(f"Errore durante la connessione al database: {e}")
        exit(1)

# Funzione per eseguire le query
def fetch_data(connection, query, params=None):
    try:
        with connection.cursor() as cursor:
            cursor.execute(query, params or ())
            return cursor.fetchall()
    except pymysql.MySQLError as e:
        logging.error(f"Errore durante l'esecuzione della query: {e}")
        return []

# Funzione per salvare i dati in CSV
def save_to_csv(df, path):
    try:
        df.to_csv(path, index=False)
        logging.info(f"Dati salvati in {path}")
    except Exception as e:
        logging.error(f"Errore durante il salvataggio del CSV: {e}")

# Funzione di elaborazione per ogni brand
def process_brand(brand_id, brand_name):
    logging.info(f"Analizzando il brand: {brand_name}")

    connection = connect_to_database()
    try:
        model_query = "SELECT id FROM device_models WHERE id_brand = %s;"
        model_ids = [row[0] for row in fetch_data(connection, model_query, (brand_id,))]

        if not model_ids:
            logging.info(f"Nessun modello trovato per il brand {brand_name}.")
            return

        model_ids_str = ','.join(map(str, model_ids))

        event_query = f"""
        SELECT 
            de.id AS event_id,
            de.id_device,
            de.date_start,
            de.date_end,
            de.variable AS event_variable,
            de.operator,
            de.reference_value,
            d.name AS device_name,
        FROM device_events de
        JOIN devices d ON de.id_device = d.id
        WHERE d.id_device_model IN ({model_ids_str})
        ORDER BY de.date_start;
        """
        events = fetch_data(connection, event_query)

        measures_dataframes = []

        for event in events:
            event_id, id_device, date_start = event[0], event[1], event[2]
            lookback_start = pd.to_datetime(date_start) - pd.Timedelta(hours=lookback_hours)

            measure_query = """
            SELECT 
                m.id AS measure_id,
                m.oid,
                m.name AS measure_name,
                m.value,
                m.value_label,
                m.measure_date,
                d.name AS device_name,
            FROM measures m
            JOIN devices d ON m.id_device = d.id
            WHERE m.id_device = %s
              AND m.measure_date BETWEEN %s AND %s
            ORDER BY m.measure_date;
            """
            measures = fetch_data(connection, measure_query, (id_device, lookback_start, date_start))

            if not measures:
                logging.info(f"Nessuna misura trovata per l'evento {event_id} nelle 24 ore precedenti.")
                continue

            measure_columns = ['measure_id', 'oid', 'measure_name', 'value', 'value_label', 'measure_date', 'device_name']
            measures_df = pd.DataFrame(measures, columns=measure_columns)

            measures_df['event_id'] = event_id
            measures_df['event_variable'] = event[4]
            measures_df['event_operator'] = event[5]
            measures_df['event_reference_value'] = event[6]

            output_csv_path = f'csv/measures_24h_before_event_{event_id}_brand_{brand_name}.csv'
            save_to_csv(measures_df, output_csv_path)
            measures_dataframes.append(measures_df)

        if measures_dataframes:
            combined_df = pd.concat(measures_dataframes, ignore_index=True)
            combined_csv_path = f'csv/measures_24h_before_events_for_brand_{brand_name}.csv'
            save_to_csv(combined_df, combined_csv_path)
        else:
            logging.info(f"Nessuna misura trovata per le 24 ore precedenti gli eventi per il brand {brand_name}.")

    except pymysql.MySQLError as e:
        logging.error(f"Errore durante l'esecuzione della query: {e}")
    finally:
        if connection and connection.open:
            connection.close()
            logging.info(f"Connessione al database chiusa per il brand {brand_name}")

def main(): 
    connection = connect_to_database()
    try:
        brand_query = "SELECT id, name FROM brands;"
        brands = fetch_data(connection, brand_query)

        with ThreadPoolExecutor(max_workers=16) as executor:
            futures = [executor.submit(process_brand, brand[0], brand[1]) for brand in brands]
            for future in as_completed(futures):
                future.result()

    except pymysql.MySQLError as e:
        logging.error(f"Errore durante l'esecuzione della query: {e}")
    finally:
        if connection and connection.open:
            connection.close()
            logging.info("Connessione al database chiusa.")

if __name__ == "__main__":
    main()



2024-07-12 02:35:25,808 - INFO - Analizzando il brand: Elenos
2024-07-12 02:35:25,809 - INFO - Analizzando il brand: Electrolink
2024-07-12 02:35:25,809 - INFO - Analizzando il brand: Itelco
2024-07-12 02:35:25,810 - INFO - Analizzando il brand: Syes
2024-07-12 02:35:25,810 - INFO - Analizzando il brand: Tem
2024-07-12 02:35:25,811 - INFO - Analizzando il brand: Profline
2024-07-12 02:35:25,811 - INFO - Analizzando il brand: 2WCOM
2024-07-12 02:35:25,811 - INFO - Analizzando il brand: ES_Telantel
2024-07-12 02:35:25,812 - INFO - Analizzando il brand: Elca
2024-07-12 02:35:25,812 - INFO - Analizzando il brand: Axel
2024-07-12 02:35:25,812 - INFO - Analizzando il brand: Elber
2024-07-12 02:35:25,812 - INFO - Analizzando il brand: Qnap
2024-07-12 02:35:25,813 - INFO - Analizzando il brand: Db Elettronica
2024-07-12 02:35:25,814 - INFO - Analizzando il brand: Deva
2024-07-12 02:35:25,814 - INFO - Analizzando il brand: Comap
2024-07-12 02:35:25,814 - INFO - Analizzando il brand: Tinycontrol

In [3]:
import pandas as pd
import os

# Definisci il percorso della cartella contenente i file CSV
cartella_csv = 'csv'  # Sostituisci con il percorso effettivo

# Lista dei file CSV nella cartella
file_csv = [f for f in os.listdir(cartella_csv) if f.endswith('.csv')]

# Inizializza una lista per contenere i DataFrame
dataframes = []

# Carica ciascun CSV in un DataFrame, elimina le colonne specifiche e aggiungilo alla lista
for file in file_csv:
    df = pd.read_csv(os.path.join(cartella_csv, file))
    # Elimina le colonne indesiderate se esistono
    columns_to_drop = ['device_name', 'oid', 'ip_address']
    df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)
    dataframes.append(df)

# Unisci tutti i DataFrame in un unico DataFrame
df_unificato = pd.concat(dataframes, ignore_index=True)

# Salva il DataFrame unificato in un unico file CSV
df_unificato.to_csv('csv/data_pre_processing.csv', index=False)  # Sostituisci con il percorso effettivo

print("File CSV unificato creato con successo.")


/var/folders/b1/pzyxnwz141n6djyl6d6kc8vr0000gn/T/ipykernel_7996/1392556416.py:15: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(cartella_csv, file))


File CSV unificato creato con successo.


In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Carica i dati dal file CSV
data = pd.read_csv('csv/data_pre_processing.csv')

# Converti la colonna 'measure_date' in datetime
data['measure_date'] = pd.to_datetime(data['measure_date'])

# Imposta 'measure_date' come indice
data.set_index('measure_date', inplace=True)

# Visualizza i dati per assicurarti che siano corretti
print(data.head())

# Seleziona solo le colonne numeriche da normalizzare/standardizzare (escludendo quelle relative agli eventi)
numeric_features = data.select_dtypes(include=[np.number]).drop(columns=['event_id'])

# Normalizzazione
min_max_scaler = MinMaxScaler()
features_normalized = min_max_scaler.fit_transform(numeric_features)

# Standardizzazione
standard_scaler = StandardScaler()
features_standardized = standard_scaler.fit_transform(numeric_features)

# Converti gli array risultanti in DataFrame per una migliore visualizzazione
features_normalized_df = pd.DataFrame(features_normalized, index=numeric_features.index, columns=numeric_features.columns)
features_standardized_df = pd.DataFrame(features_standardized, index=numeric_features.index, columns=numeric_features.columns)

# Aggiungi nuovamente le colonne non numeriche e quelle degli eventi ai DataFrame normalizzati/standardizzati
normalized_data = pd.concat([features_normalized_df, data.select_dtypes(exclude=[np.number])], axis=1)
standardized_data = pd.concat([features_standardized_df, data.select_dtypes(exclude=[np.number])], axis=1)

# Visualizza i DataFrame risultanti
print("Normalized Data:")
print(normalized_data.head())

print("Standardized Data:")
print(standardized_data.head())

# Salva i DataFrame risultanti per un'ulteriore analisi o per l'addestramento del modello
normalized_data.to_csv('csv/normalized_data.csv', index=False)
standardized_data.to_csv('csv/standardized_data.csv', index=False)


/var/folders/b1/pzyxnwz141n6djyl6d6kc8vr0000gn/T/ipykernel_7996/2903003467.py:6: DtypeWarning: Columns (2,3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('csv/data_pre_processing.csv')


                     measure_id         measure_name value value_label  \
measure_date                                                             
2024-06-24 17:02:40    89140703  networkAvailability   1.0         1.0   
2024-06-24 17:02:40    89140704      generalStatus.0   3.0         3.0   
2024-06-24 17:02:40    89140705     reflectedPower.0   0.0         0.0   
2024-06-24 17:02:40    89140706      rfTemperature.0  44.8        44.8   
2024-06-24 17:02:40    89140707             plus5v.0   5.0         5.0   

                     event_id event_variable event_operator  \
measure_date                                                  
2024-06-24 17:02:40       265   forwardPower          lower   
2024-06-24 17:02:40       265   forwardPower          lower   
2024-06-24 17:02:40       265   forwardPower          lower   
2024-06-24 17:02:40       265   forwardPower          lower   
2024-06-24 17:02:40       265   forwardPower          lower   

                     event_reference_va

In [ ]:
SELECT d.*
FROM `tx-control-seritel`.`txControl``.devices d
JOIN `tx-control-seritel``.txControl.device_models dm ON d.id_device_model = dm.ID
WHERE dm.id_device_type = 1;

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Carica i dati
data = pd.read_csv('csv/data_pre_processing.csv', dtype={
    'value': str,
    'value_label': str,
    'event_variable': str
})

# Converti le colonne numeriche
data['value'] = pd.to_numeric(data['value'], errors='coerce')
data['value_label'] = pd.to_numeric(data['value_label'], errors='coerce')

# Stampa le colonne del DataFrame
print("Colonne nel DataFrame:")
print(data.columns)

# Stampa le prime righe del DataFrame
print("\nPrime righe del DataFrame:")
print(data.head())

# Converti la colonna 'measure_date' in datetime
data['measure_date'] = pd.to_datetime(data['measure_date'])

# Se 'device_id' non è presente, creiamo un ID univoco
if 'device_id' not in data.columns:
    print("ATTENZIONE: Nessuna colonna 'device_id' trovata. Creazione di un ID univoco per ogni riga.")
    data['device_id'] = range(len(data))

# Ordina i dati per device e data
data = data.sort_values(['device_id', 'measure_date'])

# Imposta 'measure_date' come indice
data.set_index('measure_date', inplace=True)

# Funzione per creare features dalle misure precedenti
def create_features(group):
    group['target'] = group['event_id'].shift(-1).notna().astype(int)
    
    numeric_columns = group.select_dtypes(include=[np.number]).columns
    for col in numeric_columns:
        if col not in ['device_id', 'event_id', 'target']:
            group[f'{col}_mean_24h'] = group[col].rolling('24h', min_periods=1).mean()
            group[f'{col}_std_24h'] = group[col].rolling('24h', min_periods=1).std()
            group[f'{col}_min_24h'] = group[col].rolling('24h', min_periods=1).min()
            group[f'{col}_max_24h'] = group[col].rolling('24h', min_periods=1).max()
    
    return group

# Applica la funzione create_features a ogni dispositivo
data = data.groupby('device_id').apply(create_features).reset_index()

# Gestione dei valori NaN
data = data.fillna(data.mean())

# Seleziona le colonne numeriche per il modello
feature_columns = data.select_dtypes(include=[np.number]).columns.tolist()
feature_columns = [col for col in feature_columns if col not in ['device_id', 'event_id', 'target']]

# Standardizza le features
scaler = StandardScaler()
X = scaler.fit_transform(data[feature_columns])
y = data['target']

# Dividi i dati in set di training e test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape of training set:", X_train.shape)
print("Shape of test set:", X_test.shape)
print("Number of positive samples in training set:", sum(y_train))
print("Number of positive samples in test set:", sum(y_test))

Colonne nel DataFrame:
Index(['measure_id', 'measure_name', 'value', 'value_label', 'measure_date',
       'event_id', 'event_variable', 'event_operator',
       'event_reference_value'],
      dtype='object')

Prime righe del DataFrame:
   measure_id         measure_name  value  value_label         measure_date  \
0    89140703  networkAvailability    1.0          1.0  2024-06-24 17:02:40   
1    89140704      generalStatus.0    3.0          3.0  2024-06-24 17:02:40   
2    89140705     reflectedPower.0    0.0          0.0  2024-06-24 17:02:40   
3    89140706      rfTemperature.0   44.8         44.8  2024-06-24 17:02:40   
4    89140707             plus5v.0    5.0          5.0  2024-06-24 17:02:40   

   event_id event_variable event_operator  event_reference_value  
0       265   forwardPower          lower                 1000.0  
1       265   forwardPower          lower                 1000.0  
2       265   forwardPower          lower                 1000.0  
3       265   forwa

: 

# Modello di manutenzione predittiva

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Crea e addestra il modello
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Fai predizioni sul set di test
y_pred = rf_model.predict(X_test)

# Valuta il modello
print(classification_report(y_test, y_pred))

# Crea una matrice di confusione
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d')
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

# Visualizza l'importanza delle features
feature_importance = pd.DataFrame({'feature': feature_columns, 'importance': rf_model.feature_importances_})
feature_importance = feature_importance.sort_values('importance', ascending=False)
plt.figure(figsize=(10,7))
sns.barplot(x='importance', y='feature', data=feature_importance.head(20))
plt.title('Top 20 Most Important Features')
plt.show()

trasformazione dei dati temporali in un formato che può essere utilizzato per l'addestramento di un modello di machine learning
Caricamento e Pre-elaborazione dei Dati:
- Converti la colonna measure_date in un formato datetime.
- Imposta la colonna measure_date come indice.

Creazione di Finestre Temporali:

- Definisci la dimensione della finestra temporale (ad esempio, 5 misurazioni).
- Crea le finestre temporali utilizzando tecniche come lo sliding window o l'aggregazione su intervalli di tempo specifici.

# Passaggi per la Pre-elaborazione e l'Unione dei Dati
### Caricamento dei Dati dai CSV:

Carica ciascun CSV in un DataFrame separato utilizzando pandas.
### Unione dei Dati:

Unisci i DataFrame in base a chiavi comuni (ad esempio, timestamp, ID del dispositivo) per creare un DataFrame unificato.
### Preprocessing dei Dati:

Gestisci i valori mancanti.
Normalizza o standardizza le caratteristiche se necessario.
Effettua il one-hot encoding per le variabili categoriche (es. tipo di device, modello, brand).
### Feature Engineering:

Crea nuove caratteristiche che potrebbero essere utili per il modello.



In [ ]:
# import pandas as pd
# import os

# # Definisci il percorso della cartella contenente i file CSV
# cartella_csv = 'csv'  # Sostituisci con il percorso effettivo

# # Lista dei file CSV nella cartella
# file_csv = [f for f in os.listdir(cartella_csv) if f.endswith('.csv')]

# # Inizializza una lista per contenere i DataFrame
# dataframes = []

# # Carica ciascun CSV in un DataFrame e aggiungilo alla lista
# for file in file_csv:
#     df = pd.read_csv(os.path.join(cartella_csv, file))
#     dataframes.append(df)

# # Unisci tutti i DataFrame in un unico DataFrame
# df_unificato = pd.concat(dataframes, ignore_index=True)

# # Salva il DataFrame unificato in un unico file CSV
# df_unificato.to_csv('csv/data_pre_processing.csv', index=False)  # Sostituisci con il percorso effettivo

# print("File CSV unificato creato con successo.")


# Addestramento modello
## Classificazione dei dati

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Carica i dati dal file CSV
data = pd.read_csv('csv/data_pre_processing.csv')

# Converti la colonna 'measure_date' in datetime
data['measure_date'] = pd.to_datetime(data['measure_date'])

# Imposta 'measure_date' come indice
data.set_index('measure_date', inplace=True)

# Seleziona solo le colonne numeriche da normalizzare/standardizzare (escludendo quelle relative agli eventi)
numeric_features = data.select_dtypes(include=[np.number]).drop(columns=['event_id'])

# Normalizzazione
scaler = MinMaxScaler()
features_normalized = scaler.fit_transform(numeric_features)

# Creazione del DataFrame normalizzato
features_normalized_df = pd.DataFrame(features_normalized, index=numeric_features.index, columns=numeric_features.columns)

# Aggiungi nuovamente le colonne degli eventi ai DataFrame normalizzati
normalized_data = pd.concat([features_normalized_df, data[['event_id', 'event_variable', 'event_operator', 'event_reference_value']]], axis=1)

# Seleziona le feature (X) e il target (y)
X = features_normalized_df
y = data['event_id']

# Dividi i dati in set di addestramento e test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inizializza e addestra il modello di Random Forest
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

# Effettua le previsioni sul set di test
y_pred = classifier.predict(X_test)

# Valuta le prestazioni del modello
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Stampa le metriche di valutazione
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

# Stampa il classification report
print(classification_report(y_test, y_pred))


/var/folders/b1/pzyxnwz141n6djyl6d6kc8vr0000gn/T/ipykernel_7996/2731634432.py:9: DtypeWarning: Columns (2,3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('csv/data_pre_processing.csv')


# Addestramento con normalizzazione

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Carica i dati dal file CSV
data = pd.read_csv('csv/data_pre_processing.csv')

# Converti la colonna 'measure_date' in datetime
data['measure_date'] = pd.to_datetime(data['measure_date'])

# Imposta 'measure_date' come indice
data.set_index('measure_date', inplace=True)

# Seleziona solo le colonne numeriche da normalizzare/standardizzare (escludendo quelle relative agli eventi)
numeric_features = data.select_dtypes(include=[np.number]).drop(columns=['event_id'])

# Normalizzazione
scaler = MinMaxScaler()
features_normalized = scaler.fit_transform(numeric_features)

# Creazione del DataFrame normalizzato
features_normalized_df = pd.DataFrame(features_normalized, index=numeric_features.index, columns=numeric_features.columns)

# Aggiungi nuovamente le colonne degli eventi ai DataFrame normalizzati
normalized_data = pd.concat([features_normalized_df, data[['event_id', 'event_variable', 'event_operator', 'event_reference_value']]], axis=1)

# Seleziona le feature (X) e il target (y)
X = features_normalized_df
y = data['event_id']

# Dividi i dati in set di addestramento e test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inizializza e addestra il modello di Random Forest
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

# Effettua le previsioni sul set di test
y_pred = classifier.predict(X_test)

# Valuta le prestazioni del modello
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Stampa le metriche di valutazione
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

# Stampa il classification report
print(classification_report(y_test, y_pred))


/var/folders/b1/pzyxnwz141n6djyl6d6kc8vr0000gn/T/ipykernel_21441/2731634432.py:9: DtypeWarning: Columns (2,3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('csv/data_pre_processing.csv')


Accuracy: 0.7931789476868624
Precision: 0.7957893516389829
Recall: 0.7931789476868624
F1-score: 0.7932700905169303
              precision    recall  f1-score   support

          99       0.09      0.38      0.14        13
         101       0.08      0.03      0.05      1086
         105       0.10      0.05      0.06      1844
         108       0.09      0.04      0.06      2144
         109       0.14      0.18      0.16      2424
         110       1.00      1.00      1.00      2432
         111       0.14      0.36      0.20      2483
         112       0.11      0.04      0.06      2889
         113       0.26      0.25      0.25      2883
         117       1.00      1.00      1.00     11319
         118       0.73      0.83      0.78      2647
         119       0.96      0.93      0.95     11705
         120       0.49      0.51      0.50     26623
         121       0.27      0.25      0.26     26362
         122       0.12      0.14      0.13     25917
         123       0

# Isolation Forest

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Carica i dati dal file CSV
data = pd.read_csv('csv/data_pre_processing.csv')

# Converti la colonna 'measure_date' in datetime
data['measure_date'] = pd.to_datetime(data['measure_date'])

# Imposta 'measure_date' come indice
data.set_index('measure_date', inplace=True)

# Seleziona solo le colonne numeriche da normalizzare/standardizzare (escludendo quelle relative agli eventi)
numeric_features = data.select_dtypes(include=[np.number]).drop(columns=['event_id'])

# Normalizzazione
scaler = MinMaxScaler()
features_normalized = scaler.fit_transform(numeric_features)

# Creazione del DataFrame normalizzato
features_normalized_df = pd.DataFrame(features_normalized, index=numeric_features.index, columns=numeric_features.columns)

# Encoding delle variabili categoriche
label_encoder = LabelEncoder()
data['event_variable'] = label_encoder.fit_transform(data['event_variable'].astype(str))
data['event_operator'] = label_encoder.fit_transform(data['event_operator'].astype(str))

# Aggiungi nuovamente le colonne degli eventi ai DataFrame normalizzati
normalized_data = pd.concat([features_normalized_df, data[['event_id', 'event_variable', 'event_operator', 'event_reference_value']]], axis=1)

# Seleziona le feature (X) e il target (y)
X = normalized_data.drop(columns=['event_id'])
y = data['event_id']

# Dividi i dati in set di addestramento e test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inizializza e addestra il modello di Random Forest
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

# Effettua le previsioni sul set di test
y_pred = classifier.predict(X_test)

# Valuta le prestazioni del modello
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Stampa le metriche di valutazione
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

# Stampa il classification report
print(classification_report(y_test, y_pred))

# Implementazione della IsolationForest per il rilevamento delle anomalie
iso_forest = IsolationForest(contamination=0.01, random_state=42)
iso_forest.fit(X_train)

# Predizione delle anomalie
y_pred_train = iso_forest.predict(X_train)
y_pred_test = iso_forest.predict(X_test)

# Le predizioni saranno -1 per le anomalie e 1 per i punti normali
print("Anomalie nel set di addestramento:", (y_pred_train == -1).sum())
print("Anomalie nel set di test:", (y_pred_test == -1).sum())


/var/folders/b1/pzyxnwz141n6djyl6d6kc8vr0000gn/T/ipykernel_21441/231777342.py:9: DtypeWarning: Columns (2,3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('csv/data_pre_processing.csv')


Accuracy: 0.7898495723134402
Precision: 0.7922386652244102
Recall: 0.7898495723134402
F1-score: 0.789854879599532
              precision    recall  f1-score   support

          99       0.06      0.23      0.10        13
         101       0.07      0.04      0.05      1086
         105       0.08      0.04      0.06      1844
         108       0.07      0.03      0.04      2144
         109       0.13      0.10      0.11      2424
         110       1.00      1.00      1.00      2432
         111       0.14      0.39      0.20      2483
         112       0.11      0.05      0.07      2889
         113       0.24      0.25      0.24      2883
         117       1.00      1.00      1.00     11319
         118       0.77      0.88      0.82      2647
         119       0.97      0.94      0.96     11705
         120       0.49      0.50      0.49     26623
         121       0.26      0.23      0.25     26362
         122       0.10      0.10      0.10     25917
         123       0.